This function processes data for multiple countries. It reads input CSV files, drops unnecessary columns, filters rows based on a condition, calculates totals, removes duplicates, reads another CSV file, and merges the data. 

In [1]:
import pandas as pd
import os

def process_country_data():

    countries = ['SP', 'UK', 'DE', 'DK', 'SE', 'HU', 'IT', 'PO', 'NE']

    for country in countries:
        file_path = os.path.join('../data', f'total_green_{country}.csv')
        df = pd.read_csv(file_path, low_memory=False)

        # Dropping unnecessary columns
        columns_to_drop = ['EndTime', 'UnitName', 'AreaID']
        df = df.drop(columns=columns_to_drop)

        # Defining the output CSV file name and a common column
        load_csv_file = f'../data/load_{country}.csv'
        common_column = 'StartTime'

        # Condition to filter rows based on 'PsrType'
        condition = df['PsrType'].isin(['B01','B09', 'B10', 'B11', 'B12', 'B13', 'B15', 'B16', 'B18', 'B19'])
        
        filtered_df = df[condition].copy()
    
        # Calculating the total green energy  
        filtered_df['total'] = filtered_df.groupby('StartTime')['quantity'].transform('sum')

        # Removing duplicate rows after combining total green energy
        filtered_df = filtered_df.drop_duplicates(subset='StartTime') 

        # Merging load energy with total green energy for each country
        load_df = pd.read_csv(load_csv_file)
        load_df = load_df.drop(columns=columns_to_drop)
        load_df = pd.merge(load_df, filtered_df, on=common_column, how='left')
        
        output_path = '../data'
        load_df.to_csv(f'{output_path}/total_{country}.csv', index=False)

    return

In [2]:
process_country_data()

This function normalizes data for multiple countries.  It homogenizes date stamps to 1-hour intervals for consistency.

In [3]:
def normalize_by_hour():
    
    countries = ['SP', 'UK', 'DE', 'DK', 'SE', 'HU', 'IT', 'PO', 'NE']

    for country in countries:
        
        file_path = os.path.join('../data', f'total_{country}.csv')
        df = pd.read_csv(file_path, low_memory=False)

        # Removing unnecessary parts and converting to to datetime format
        df['StartTime'] = df['StartTime'].str[:-7].str.replace('T', ' ')
        df['StartTime'] =  pd.to_datetime(df['StartTime'])
    
        df.set_index('StartTime', inplace=True)

        # Resampling the DataFrame to hourly frequency
        df['total'] = df.resample('1H').sum()['total']
        df['load_norm'] = df.resample('1H').sum()['Load']
    
        df = df.reset_index()

        # Filtering rows where the minute part of 'StartTime' is 0
        df = df[df['StartTime'].dt.minute == 0]
    
        df.rename(columns={'total': f'green_energy_{country}', 
                         'load_norm': f'{country}_Load'}, inplace=True)    
    
         # Dropping unnecessary columns
        columns_to_drop = ['Load', 'PsrType', 'quantity' ]
        df = df.drop(columns=columns_to_drop)       

        df['StartTime'] = pd.to_datetime(df['StartTime'])
        output_path = '../data'
        df.to_csv(f'{output_path}/norm_{country}.csv', index=False)

    return
    

In [4]:
normalize_by_hour()

This function merges normalized data from multiple countries into a single DataFrame. It reads normalized CSV files, merges them based on the 'StartTime' column using an outer join, interpolates missing values using linear interpolation, and writes the merged data to a new CSV file named 'merged_data.csv'.

In [5]:
def merge_files():

    countries = ['SP', 'UK', 'DE', 'DK', 'SE', 'HU', 'IT', 'PO', 'NE']

    for country in countries:
        
        # Initializing an empty DataFrame for the result
        result_df = pd.DataFrame()
    
        for country in countries:
            file_path = os.path.join('../data', f'norm_{country}.csv')
            df = pd.read_csv(file_path)
            
            # Assigning results to the first DataFrame
            if result_df.empty:
                result_df = df
            else:
                # Merging the current DataFrame with the result DataFrame on the 'StartTime' column
                result_df = pd.merge(result_df, df, on='StartTime', how = 'outer')
        
        result_df.set_index('StartTime', inplace=True, drop = True) 
                
        result_df.interpolate(method='linear', limit_direction='both', inplace=True)
    
        output_path = '../data'
        result_df.to_csv(f'{output_path}/merged_data.csv', index = True)

    return

In [6]:
merge_files()

This function creates a separate file for each country containing surplus of green energy and country code.

In [7]:
def normalize_surplus():
    
    countries = ['SP', 'UK', 'DE', 'DK', 'SE', 'HU', 'IT', 'PO', 'NE']

    for country in countries:
        
        file_path = os.path.join('../data', f'total_{country}.csv')
        df = pd.read_csv(file_path, low_memory=False)
    
        df['StartTime'] = df['StartTime'].str[:-7].str.replace('T', ' ')
        df['StartTime'] =  pd.to_datetime(df['StartTime'])
    
        df.set_index('StartTime', inplace=True)
    
        df['total'] = df.resample('1H').sum()['total']
        df['load_norm'] = df.resample('1H').sum()['Load']
    
        df = df.reset_index()
    
        df['StartTime'] = pd.to_datetime(df['StartTime'])
        df = df[df['StartTime'].dt.minute == 0]
    
        columns_to_drop = ['Load', 'PsrType', 'quantity' ]
        df = df.drop(columns=columns_to_drop)

        # Calculating 'surplus' and adding 'country' column
        df['surplus'] = df['total'] - df['load_norm']
        df['country'] = f'{country}'    
        output_path = '../data'
        df.to_csv(f'{output_path}/norm_surp_{country}.csv', index=False)

    return

In [8]:
normalize_surplus()

This function normalizes surplus data for multiple countries. In addition, it creates a separate column containing ID code for the country with the biggest surplus of green energy.

The The country IDs are as following
- Spain: SP 0
- United Kingdom: UK 1
- Germany: DE 2 
- Denmark: DK 3
- Sweden: SE 4
- Hungary: HU 5
- Italy: IT 6
- Poland: PO 7
- Netherlands: NE 8


In [9]:
def surplus_calc():
    
    file_names = ['SP', 'UK', 'DE', 'DK', 'SE', 'HU', 'IT', 'PO', 'NE']
    
    dfs = {}
    
    for i, country_code in enumerate(file_names):
        file_path = f'../data/norm_surp_{country_code}.csv'
        df = pd.read_csv(file_path)
        dfs[f'df{i}'] = df
    
    # Concatenating the 'surplus' columns from all DataFrames
    df_new = pd.concat([df['surplus'] for df in dfs.values()], axis=1, join='outer') 
    
    df_new.columns = [f'surplus_{i}' if i < 9 else 'max_value' for i in range(9)]
    
    max_column_index = df_new.idxmax(axis=1)
    
    # Finding the maximum value in each row
    max_value = df_new.max(axis=1)
    
    # Creating a new column to store the maximum value
    df_new['max_value'] = max_value
    
    # Creating a new column to store the location of the maximum value
    df_new['max_column'] = max_column_index
    
    start_time_column = dfs['df0']['StartTime']
    df_new['StartTime'] = start_time_column
       
    df_new['max_column'] = df_new['max_column'].str[-1].astype(int)
    df_new.head()
    df_new.to_csv('../data/surplus.csv', index=False)

    return

In [10]:
surplus_calc()

This function combines surplus data with merged data into the final data frame

In [11]:
def final_merge():
    
    df1 = pd.read_csv('../data/merged_data.csv')
    df2 = pd.read_csv('../data/surplus.csv')
    
    df = pd.merge(df1, df2[['StartTime', 'max_column']], on='StartTime', how='inner')
    df.rename(columns={'max_column': 'label'}, inplace=True)   
    
    
    df.to_csv(f'../data/final_data.csv', index=False)

    return

In [12]:
final_merge()